In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv
/kaggle/input/future-sales-prediction/prediction.csv
/kaggle/input/future-sales-prediction/future_sales_pred.ipynb


In [2]:
import datetime as dt
import logging
import warnings
warnings.filterwarnings('ignore')
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [3]:
!pip install catboost
import catboost
from catboost import CatBoostRegressor
from catboost import Pool, cv

In [4]:
### Read Data 
items=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_cat=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
train_df=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
shops=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
test_df=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
submission=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')

In [5]:
display(items.shape)
display(item_cat.shape)
display(train_df.shape)
display(shops.shape)
display(test_df.shape)
display(submission.shape)

(22170, 3)

(84, 2)

(2935849, 6)

(60, 2)

(214200, 3)

(214200, 2)

In [6]:
print(items.head())
print(item_cat.head())
print(train_df.head())
print(shops.head())
print(test_df.head())
print(submission.head())

                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3                40  
4                40  
        item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4
         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00

In [7]:
train_df.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [8]:
train_df.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [9]:
train_df.head()
## dropping the columns not required

train_df.drop(['date_block_num','item_price'], axis=1, inplace = True)

In [10]:
submission.tail()

,ID,item_cnt_month
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5
214199,214199,0.5


In [11]:
train_df['date'] = pd.to_datetime(train_df['date'])

In [12]:
train_df.head()
## Get month,year only
train_df['month']=train_df['date'].dt.month
train_df['year']=train_df['date'].dt.year

In [13]:
train_df

,date,shop_id,item_id,item_cnt_day,month,year
0,2013-02-01,59,22154,1.0,2,2013
1,2013-03-01,25,2552,1.0,3,2013
2,2013-05-01,25,2552,-1.0,5,2013
3,2013-06-01,25,2554,1.0,6,2013
4,2013-01-15,25,2555,1.0,1,2013
...,...,...,...,...,...,...
2935844,2015-10-10,25,7409,1.0,10,2015
2935845,2015-09-10,25,7460,1.0,9,2015
2935846,2015-10-14,25,7459,1.0,10,2015
2935847,2015-10-22,25,7440,1.0,10,2015


In [14]:
train_df['date'] = train_df['date'].apply(lambda x: x.strftime('%Y-%m'))

In [15]:
train_df.head()

,date,shop_id,item_id,item_cnt_day,month,year
0,2013-02,59,22154,1.0,2,2013
1,2013-03,25,2552,1.0,3,2013
2,2013-05,25,2552,-1.0,5,2013
3,2013-06,25,2554,1.0,6,2013
4,2013-01,25,2555,1.0,1,2013


In [16]:
data = train_df.groupby(['date','shop_id','item_id']).sum()


In [17]:
data = data.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)


In [18]:
data.reset_index(inplace=True)

In [19]:
data

date,shop_id,item_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12
0,0,30,0,31,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,31,0,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,32,4,6,2,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,33,2,2,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,35,2,6,0,1,0,2,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,59,22154,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424120,59,22155,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424121,59,22162,0,0,0,0,0,0,0,0,...,2,1,1,0,1,0,1,0,1,0
424122,59,22164,0,0,0,0,0,0,0,0,...,1,2,0,0,1,0,0,0,1,0


In [20]:
display(test_df.shape)
test_df=pd.merge(test_df, data, on=['shop_id','item_id'], how='left')
display(test_df.shape)
test_df = test_df.fillna(0)

(214200, 3)

(214200, 39)

In [21]:
test_df.T

,0,1,2,3,4,5,6,7,8,9,...,214190,214191,214192,214193,214194,214195,214196,214197,214198,214199
ID,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,214190.0,214191.0,214192.0,214193.0,214194.0,214195.0,214196.0,214197.0,214198.0,214199.0
shop_id,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0
item_id,5037.0,5320.0,5233.0,5232.0,5268.0,5039.0,5041.0,5046.0,5319.0,5003.0,...,3280.0,4393.0,4352.0,18049.0,18027.0,18454.0,16188.0,15757.0,19648.0,969.0
2013-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2013-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
data.T

,0,1,2,3,4,5,6,7,8,9,...,424114,424115,424116,424117,424118,424119,424120,424121,424122,424123
date,,,,,,,,,,,,,,,,,,,,,
shop_id,0,0,0,0,0,0,0,0,0,0,...,59,59,59,59,59,59,59,59,59,59
item_id,30,31,32,33,35,36,40,42,43,49,...,22144,22145,22150,22151,22153,22154,22155,22162,22164,22167
2013-01,0,0,4,2,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2013-02,31,11,6,2,6,1,0,1,0,1,...,0,0,0,1,0,1,0,0,0,0
2013-03,0,0,2,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2013-04,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2013-05,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-06,0,0,1,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-07,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [23]:
# split into train and test sets

Y_train = data['2015-12'].values
X_train = data.drop(['2015-12'], axis = 1)
X_test = test_df
print(X_train.shape, Y_train.shape)
print(X_test.shape)

(424124, 37) (424124,)
(214200, 39)


In [24]:
x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.20, random_state=123)

In [25]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(339299, 37)
(84825, 37)
(339299,)
(84825,)


In [26]:
## Prepare a pool of train and validation set

pool_train=Pool(x_train, y_train)
pool_val=Pool(x_test, y_test)

In [27]:
def modelfit(params,poolX,useTrainCV=True,cv_folds=5,early_stopping_rounds=40):
    if useTrainCV:
        cvresult = cv(params=params, pool=poolX,nfold=cv_folds,early_stopping_rounds=early_stopping_rounds,plot=True)
    return cvresult

In [28]:
## Preparing  cv class
params={
    'loss_function':'RMSE'
}

In [29]:
n_est=modelfit(params,pool_train)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6049355	test: 0.5837142	best: 0.5837142 (0)
1:	learn: 0.5979226	test: 0.5788520	best: 0.5788520 (1)
2:	learn: 0.5913296	test: 0.5743119	best: 0.5743119 (2)
3:	learn: 0.5846689	test: 0.5691687	best: 0.5691687 (3)
4:	learn: 0.5783455	test: 0.5650912	best: 0.5650912 (4)	total: 1.11s	remaining: 3m 40s
5:	learn: 0.5726625	test: 0.5611686	best: 0.5611686 (5)
6:	learn: 0.5665645	test: 0.5568480	best: 0.5568480 (6)
7:	learn: 0.5608576	test: 0.5530967	best: 0.5530967 (7)
8:	learn: 0.5550238	test: 0.5494137	best: 0.5494137 (8)
9:	learn: 0.5495119	test: 0.5463676	best: 0.5463676 (9)
10:	learn: 0.5445628	test: 0.5433683	best: 0.5433683 (10)
11:	learn: 0.5395057	test: 0.5402305	best: 0.5402305 (11)
12:	learn: 0.5340307	test: 0.5368812	best: 0.5368812 (12)
13:	learn: 0.5294220	test: 0.5342263	best: 0.5342263 (13)
14:	learn: 0.5244974	test: 0.5316215	best: 0.5316215 (14)
15:	learn: 0.5194398	test: 0.5288599	best: 0.5288599 (15)
16:	learn: 0.5149196	test: 0.5264355	best: 0.5264355 (16)
17:

In [30]:
n_est.shape[0]

664

In [31]:
# Fit the model with iteration=664
cboost1=CatBoostRegressor(iterations=664,loss_function='RMSE',random_seed=123)
cboost1.fit(x_train,y_train)

Learning rate set to 0.150313
0:	learn: 0.5767792	total: 34.2ms	remaining: 22.7s
1:	learn: 0.5447305	total: 60.8ms	remaining: 20.1s
2:	learn: 0.5153078	total: 88.6ms	remaining: 19.5s
3:	learn: 0.4993826	total: 115ms	remaining: 19s
4:	learn: 0.4784646	total: 150ms	remaining: 19.8s
5:	learn: 0.4657885	total: 184ms	remaining: 20.2s
6:	learn: 0.4534179	total: 214ms	remaining: 20.1s
7:	learn: 0.4372183	total: 242ms	remaining: 19.8s
8:	learn: 0.4196950	total: 270ms	remaining: 19.6s
9:	learn: 0.4031121	total: 298ms	remaining: 19.5s
10:	learn: 0.3915637	total: 325ms	remaining: 19.3s
11:	learn: 0.3850157	total: 352ms	remaining: 19.1s
12:	learn: 0.3716864	total: 382ms	remaining: 19.1s
13:	learn: 0.3660584	total: 410ms	remaining: 19.1s
14:	learn: 0.3543735	total: 439ms	remaining: 19s
15:	learn: 0.3495863	total: 467ms	remaining: 18.9s
16:	learn: 0.3450956	total: 493ms	remaining: 18.8s
17:	learn: 0.3367669	total: 521ms	remaining: 18.7s
18:	learn: 0.3327936	total: 577ms	remaining: 19.6s
19:	learn: 0

In [32]:
#Predict training set:
train_predictions = cboost1.predict(x_train)
#Print model report:
print("\nModel Report Train")
print("Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_train, train_predictions))
print("R^2 Score (Train): %f" % metrics.r2_score(y_train, train_predictions))


Model Report Train
Root Mean Square Error : 0.04166
R^2 Score (Train): 0.888667


In [33]:
#Predict training set:
test_predictions = cboost1.predict(x_test)
#Print model report:
print("\nModel Report Test")
print("Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_test, test_predictions))
print("R^2 Score (Train): %f" % metrics.r2_score(y_test, test_predictions))


Model Report Test
Root Mean Square Error : 0.2557
R^2 Score (Train): 0.252886


In [34]:
#Predict test set data:
test_predictions = cboost1.predict(x_test)
#Print model report:
print("\nModel Report Test")
print("Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_test, test_predictions))
print("R^2 Score (Train): %f" % metrics.r2_score(y_test, test_predictions))


Model Report Test
Root Mean Square Error : 0.2557
R^2 Score (Train): 0.252886


In [35]:

X_train

date,shop_id,item_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11
0,0,30,0,31,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,31,0,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,32,4,6,2,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,33,2,2,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,35,2,6,0,1,0,2,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,59,22154,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424120,59,22155,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424121,59,22162,0,0,0,0,0,0,0,0,...,8,2,1,1,0,1,0,1,0,1
424122,59,22164,0,0,0,0,0,0,0,0,...,1,1,2,0,0,1,0,0,0,1


In [36]:
prediction = cboost1.predict(X_test)


In [37]:
prediction = list(map(round, prediction))


In [38]:
len(prediction)

214200

In [39]:
#Submission data

submission.shape

(214200, 2)

In [40]:
submission
submission['item_cnt_month'] = prediction
submission.to_csv('prediction.csv', index=False)

In [41]:
submission.tail()

,ID,item_cnt_month
214195,214195,0
214196,214196,0
214197,214197,0
214198,214198,0
214199,214199,0
